In [56]:
from bs4 import BeautifulSoup
import os
import pandas as pd
from unidecode import unidecode
import urllib.request
from pathlib import Path
import matplotlib.pyplot as plt

In [57]:
list_review_text = {
    "contribution" : "Please describe the contribution of the paper",
    "strenghts" : "Please list the main strengths of the paper",
    "weakness" : "Please list the main weaknesses of the paper",
    "clarity" : "Please rate the clarity and organization of this paper",
    "repro" : "Please comment on the reproducibility of the paper",
    "detailed" : "Please provide detailed and constructive comments for the authors",
    "rate" : "Rate the paper on a scale of 1-8, 8 being the strongest",
    "justifictation" : "Please justify your recommendation.",
    "number of paper": "Number of papers in your stack",
    "ranking" : "What is the ranking of this paper in your review stack?",
    "confidence" : "Reviewer confidence",
    "rate rebuttal" : "[Post rebuttal] After reading the author’s rebuttal, state your overall opinion of the paper if it has been changed",
    "justification rebuttal" : "[Post rebuttal] Please justify your decision",
}

list_categories_str = [ "contribution", "strenghts", "weakness", "repro", "detailed", "justifictation" ]
list_categories_scores = ["clarity", "rate", "confidence", "rate rebuttal"]

columns_reviews = ["id", "category", "title" ,"review 1" ,"review 2" , "review 3", ]
columns_statistics = ["id", "category", "title" , "words1", "words2", "words3"]

In [58]:
# Get the list of all the html files from the year chosen
def get_accepted_paper_list(year: str = "2023"):
    """
    Get the list of all html files
    """

    miccai_website_path = "https://conferences.miccai.org"
    path_online_list = miccai_website_path + f"/{year}/papers/"

    reponse = urllib.request.urlopen(path_online_list)
    contenu_web = reponse.read().decode('UTF-8')
    soup = BeautifulSoup(contenu_web, "html.parser")

    all = soup.find_all('a')
    paper_list = [(miccai_website_path + link.get('href')) for link in all if link.get('href').endswith('html')]

    return paper_list

In [59]:
year_ = "2023"
paper_list = get_accepted_paper_list(year=year_)

In [60]:
def extract_reviews(paper, df_reviews, df_stats, year="2023",  category= "repro"):
    
    # open html webpage and extract with BeautifulSoup
    reponse = urllib.request.urlopen(paper)
    contenu_web = reponse.read().decode('UTF-8')
    soup = BeautifulSoup(contenu_web, "html.parser")

    if year == "2023" :
        paper_title = soup.find("title").get_text().rstrip("MICCAI 2023 - Accepted Papers, Reviews, Author Feedback").rstrip(' |')
    elif year == "2022" :
        paper_title = soup.find("title").get_text().rstrip("MICCAI 2022 - Accepted Papers and Reviews").rstrip(' |')

    paper_id = Path(paper).name[:13]
    text = list_review_text[category]
    repro_reviews_paragraph = soup.find_all(lambda tag: tag.name == "li" and text in tag.text)
    repro_exact_text = soup.find(lambda tag: tag.name == "strong" and text in tag.text).get_text()

    i=0
    for review in repro_reviews_paragraph[:3]:
        i +=1
        tmp_review = unidecode(review.get_text().strip(repro_exact_text))
        tmp_review = tmp_review.strip("\n")
        tmp_review = tmp_review.replace("\n          \n"," ")
        tmp_review = tmp_review.replace("\t", " ")
        tmp_review = tmp_review.replace("\n\n\n\n"," ")
        tmp_review = tmp_review.replace("\n\n\n"," ")
        tmp_review = tmp_review.replace("\n\n"," ")
        tmp_review = tmp_review.replace("\n"," ")
        tmp_review = tmp_review.replace("\t", " ")

        df_reviews.loc[(paper_id, unidecode(paper_title)), (category, f"review {i}")]=tmp_review
        df_stats.loc[(paper_id, unidecode(paper_title)), (category, f"review {i}")]=len(str(tmp_review).split())



In [61]:

def extract_reproducibility_paragraph(paper_list, year= "2023"):
    # Extract the reproducibility paragraph from the reviews and export as csv file
    iterables = [list_categories_str, ["review 1", "review 2", "review 3"]]

    index_line = pd.MultiIndex.from_product(iterables, names=["category", "review"])
    index_column = pd.MultiIndex.from_product( [[], []], names=["id", "title"])

    df_reviews = pd.DataFrame(index = index_column, columns=index_line)
    df_stats = pd.DataFrame(index = index_column, columns=index_line)

    for paper in paper_list:
        for category in list_categories_str:
             
            extract_reviews(paper, df_reviews, df_stats, year, category)

    return df_reviews, df_stats

In [62]:
# create th output directory 
output_directory = Path(f"../miccai{year_}/")
if not output_directory.is_dir():
    os.mkdir(output_directory)
path_all_reviews = output_directory / 'all_reviews.csv'
path_all_stats = output_directory / 'count_words.csv'

In [63]:
# extract 33 min  
if (not path_all_reviews.is_file()) or (not path_all_stats.is_file()):

    print(f"Extract reviews and count word for year {year_}")
    df_all_reviews, df_all_stats = extract_reproducibility_paragraph(paper_list, year_)

    df_all_reviews.to_csv(path_all_reviews, index = True, sep="\t", encoding='utf-8')
    df_all_stats.to_csv(path_all_stats, index = True, sep="\t", encoding='utf-8')

else:
    import pandas as pd
    print(f"Import tsv from {output_directory}")

    df_all_reviews = pd.read_csv(path_all_reviews, sep= "\t",  header=[0, 1], index_col=[0,1], skip_blank_lines=True)
    df_all_stats = pd.read_csv(path_all_stats, sep= "\t",  header=[0, 1], index_col=[0,1], skip_blank_lines=True)


Extract reviews and count word for year 2023


In [64]:
def get_repro_copy_paste(df_all_reviews, output_directory= "../results", threshold:int  = 10 ):

    from copy import copy
    df_all_reviews_wo_copy_paste = copy(df_all_reviews)
    df_bad_reviews = pd.DataFrame(columns=columns_reviews)
    df_bad_reviews.set_index(["id", "category"], inplace= True)
    
    for id, id_df in df_all_reviews.groupby(level=0):
        for _, title in id_df.index.values:
            for category in list_categories_str:
                if category != "repro" :
                    for i in range(1,3):
                        repro = id_df.loc[(id,title), ("repro", f"review {i}")]
                        cate = id_df.loc[(id, title), (category, f"review {i}")]
                        if len(str(repro)) > threshold : 
                            if str(repro) in str(cate):
                                df_bad_reviews.loc[(id, title), (category, f"review {i}")] = id_df.loc[(id, title), (category, f"review {i}")]
                                try :
                                    df_all_reviews_wo_copy_paste.drop((id, title), inplace=True)
                                except:
                                    pass

        
    df_all_reviews_wo_copy_paste.to_csv(os.path.join(output_directory ,f'all_reviews_wo_copy_paste.csv'), index = True, sep="\t", encoding='utf-8')
    df_bad_reviews.to_csv(os.path.join(output_directory ,f'copy_paste_reviews_{threshold}.csv'), index = True, sep="\t", encoding='utf-8')


In [65]:
# Get the list of review where the reviewer had copy/paste the review repro from another review
get_repro_copy_paste(df_all_reviews=df_all_reviews, output_directory=output_directory)


/var/folders/03/3_l2qbvj5sjf070rq3vks9rc001396/T/ipykernel_47235/1742350489.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'The code is not provided. The method is built upon the training segmentation label.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_bad_reviews.loc[(id, title), (category, f"review {i}")] = id_df.loc[(id, title), (category, f"review {i}")]
/var/folders/03/3_l2qbvj5sjf070rq3vks9rc001396/T/ipykernel_47235/1742350489.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Insufficient Dataset Description: The authors have not provided an adequate description of the dataset utilized in the study, which may hinder the reader's understanding of the context and the applicability of the proposed method. Furthermore, the lack of information regarding the preprocessing steps implement

In [66]:
def count_checklist(df_all_reviews, output_directory= "../results", category = "repro"):

    df_checklist  = pd.DataFrame(columns=columns_reviews)
    df_checklist.set_index(["id", "category"], inplace= True)
    for id, id_df in df_all_reviews.groupby(level=0):
        for _, title in id_df.index.values:
            for i in range(1,4):
                review = str(df_all_reviews.loc[(id, title), (category, f"review {i}")])

                if ("check-list" in review) or ("checklist" in review) or ("check list" in review):
                    df_checklist.loc[(id, title), (category, f"review {i}")] = df_all_reviews.loc[(id, title), (category, f"review {i}")]

    df_checklist.to_csv(os.path.join(output_directory ,f'nb_checklist_{category}.csv'), index = True, sep="\t", encoding='utf-8')


In [67]:
# Count the number of occurrences of the word checklist in the repro reviews
count_checklist(df_all_reviews=df_all_reviews, output_directory=output_directory, category="repro")

/var/folders/03/3_l2qbvj5sjf070rq3vks9rc001396/T/ipykernel_47235/1742228500.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Reproducibility should be possible, as training and evaluation code is provided (based on the checklist).' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_checklist.loc[(id, title), (category, f"review {i}")] = df_all_reviews.loc[(id, title), (category, f"review {i}")]


In [68]:
def create_rating_excel(df_all_reviews, output_directory="../results"):
    df_repro_excel = df_all_reviews.loc[:, ("repro")]
    df_repro_excel.to_excel(os.path.join('../rating' ,f'reviwes_repro_{year_}.xlsx'))

In [69]:
# Save the excel file with the repro reviews to create the rating file
create_rating_excel(df_all_reviews= df_all_reviews, output_directory= output_directory)

In [70]:
# TODO 

# pas de resultats dans ce notebook


# spliter les fonctions 
# print plus de trucs pour que ca soit plus beau et lisible 


# dans rating_analysisn 
# correler la longueur de la review repro a la longueur totale
# Donner des examples au hasard
#   - entre 40 et 60 centiles
#   - entre 0 et 30 centiles
# extraire les scores !!!
